In [1]:
from openff.pdbscan.polars import COORD_LINE_SCHEMA as schema

schema

{'id': String,
 'err': String,
 'lineNo': Int32,
 'serial': Int32,
 'name': String,
 'altLoc': String,
 'resName': String,
 'chainID': String,
 'resSeq': Int32,
 'iCode': String,
 'x': Float32,
 'y': Float32,
 'z': Float32,
 'occupancy': Float32,
 'tempFactor': Float32,
 'element': String,
 'charge': String,
 'terminated': Boolean,
 'modelNo': Int32,
 'conects': List(Int32)}

In [3]:
import polars as pl

all_ids = (
    pl.scan_parquet("/home/joshmitchell/Downloads/pdb/coords_parquet/coords_*.parquet")
    .select(pl.col("id"))
    .unique()
    .collect(streaming=True)
)

In [17]:
import numpy as np

all_ids_shuffled = np.array(all_ids['id'])
np.random.shuffle(all_ids_shuffled)

In [19]:
with open("../../pdb-census/pdbfix/all_pdb_ids_shuffled.txt", 'w') as f:
    for pdbid in all_ids_shuffled:
        print(pdbid, file=f)

In [2]:
import polars as pl

protein_residues = [
    "ALA",
    "ARG",
    "ASN",
    "ASP",
    "CYS",
    "GLU",
    "GLN",
    "GLY",
    "HIS",
    "ILE",
    "LEU",
    "LYS",
    "MET",
    "PHE",
    "PRO",
    "SER",
    "THR",
    "TRP",
    "TYR",
    "VAL",
]
ids_with_hets = (
    pl.scan_parquet("/home/joshmitchell/Downloads/pdb/coords_parquet/coords_*.parquet")
    .filter(pl.col("resName").is_in(protein_residues).not_())
    .select(pl.col("id"))
    .unique()
    .collect(streaming=True)
)

In [3]:
len(ids_with_hets)

197263

In [4]:
ids_without_hets = (
    pl.scan_parquet("/home/joshmitchell/Downloads/pdb/coords_parquet/coords_*.parquet")
    .select(pl.col("id"))
    .unique()
    .filter(pl.col("id").is_in(ids_with_hets).not_())
    .collect(streaming=True)
)
ids_without_hets = set(tup[0] for tup in ids_without_hets.iter_rows())
len(ids_without_hets)

18272

In [5]:
from pathlib import Path

protein_only_pdbs = {
    path
    for path in Path("/home/joshmitchell/Downloads/pdb").glob("*/*.ent.gz")
    if path.stem[3:7] in ids_without_hets
}
assert len(protein_only_pdbs) == len(ids_without_hets)

In [6]:
from openff.units import unit

time_per_pdb = 30 * unit.second
pdbs_at_a_time = 24
(len(ids_without_hets) * time_per_pdb / pdbs_at_a_time).to(unit.hour)

6.344444444444444 <Unit('hour')>

In [7]:
import gzip
from itertools import batched
from multiprocessing import Pool
from typing import Any, Callable, Iterable, TextIO

from openff.toolkit import Topology

TEST_PDB_LOADER_SCHEMA = {
    "id": pl.datatypes.String,
    "err": pl.datatypes.String,
    "n_molecules": pl.datatypes.UInt64,
    "n_atoms": pl.datatypes.UInt64,
    "n_hydrogens": pl.datatypes.UInt64,
}


def proc_batch(batch, schema, pdb_loader):
    data = {k: [] for k in schema}
    for path in batch:
        for column in data.values():
            column.append(None)
        data["id"][-1] = path.stem[3:-4]
        try:
            with gzip.open(path, "rt") as f:
                topology = pdb_loader(f)
        except Exception as e:
            data["err"][-1] = repr(e)
            continue
        data["n_molecules"][-1] = topology.n_molecules
        data["n_atoms"][-1] = topology.n_atoms
        data["n_hydrogens"][-1] = sum(
            1 for atom in topology.atoms if atom.atomic_number == 1
        )

    return data


def test_pdb_loader(
    *,
    pdb_loader: Callable[[TextIO], Topology],
    chunk_size: int,
    threads: int,
    paths: Iterable[Path],
) -> pl.DataFrame:
    df = pl.DataFrame(schema=TEST_PDB_LOADER_SCHEMA)
    try:
        for i, batch in enumerate(batched(paths, n=chunk_size * threads)):
            print("batch", i)
            with Pool(threads) as pool:
                batch_results = list(
                    pool.imap_unordered(
                        partial(
                            proc_batch, schema=TEST_PDB_LOADER_SCHEMA, pdb_loader=pdb_loader
                        ),
                        batched(batch, chunk_size),
                    )
                )
    
            print("concatenating")
            df = pl.concat(
                [df, *(pl.DataFrame(d, schema=TEST_PDB_LOADER_SCHEMA) for d in batch_results)]
            )
            print("rechunking")
            df.rechunk()
    except KeyboardInterrupt:
        print("exiting early")
        return df

    return df

In [30]:
from functools import partial

from openff.pdbscan.pdb import topology_from_pdb

custom_pdb_loader_df = test_pdb_loader(
    pdb_loader=partial(topology_from_pdb, replace_missing_atoms=True),
    chunk_size=100,
    threads=24,
    paths=list(protein_only_pdbs),
)

batch 0
concatenating
rechunking
batch 1
concatenating
rechunking
batch 2
concatenating
rechunking
batch 3
concatenating
rechunking
batch 4
concatenating
rechunking
batch 5
concatenating
rechunking
batch 6
concatenating
rechunking
batch 7
concatenating
rechunking


In [31]:
custom_pdb_loader_df

id,err,n_molecules,n_atoms,n_hydrogens
str,str,u64,u64,u64
"""7saz""",null,13,11348,5627
"""3mv2""",null,11,28254,13973
"""2lyc""","""NoMatchingResidueDefinitionErr…",null,null,null
"""5hm1""",null,11,6045,2947
"""6bzk""","""NoMatchingResidueDefinitionErr…",null,null,null
…,…,…,…,…
"""7v1m""",null,15,16214,8019
"""2dj0""","""NoMatchingResidueDefinitionErr…",null,null,null
"""2irt""",null,3,4481,2186


In [32]:
custom_pdb_loader_df.write_parquet("peptideonly_custom_pdb_loader_df.parquet")

In [42]:
errors = custom_pdb_loader_df.filter(pl.col("err").is_not_null()).select(["id", "err"])
errors

id,err
str,str
"""2lyc""","""NoMatchingResidueDefinitionErr…"
"""6bzk""","""NoMatchingResidueDefinitionErr…"
"""2jqz""","""NoMatchingResidueDefinitionErr…"
"""2m86""","""NoMatchingResidueDefinitionErr…"
"""2lnb""","""NoMatchingResidueDefinitionErr…"
…,…
"""7r9e""","""NoMatchingResidueDefinitionErr…"
"""2k9z""","""NoMatchingResidueDefinitionErr…"
"""1n02""","""NoMatchingResidueDefinitionErr…"


In [43]:
errors[0, 0], errors[0, 1]

('2lyc',
 'NoMatchingResidueDefinitionError("No residue definitions covered all atoms in MET#1:A\\nThe following definitions were considered:\\n    0: [(\'H1\', 8)] were not among HG3 (or its synonym 2HG), HG2 (or its synonym 1HG), H2 (or its synonym HN2), CA, HB2 (or its synonym 1HB), HA, HE2 (or its synonym 2HE), H, C, SD, OXT, HXT, O, CE, HE3 (or its synonym 3HE), CG, HE1 (or its synonym 1HE), CB, N, HB3 (or its synonym 2HB)")')

In [48]:
from openff.toolkit import Topology

pdb_2lyc = Topology.from_pdb("/home/joshmitchell/Downloads/2lyc.pdb")

/home/joshmitchell/Documents/openff/openff-pdbscan/.soap/test/lib/python3.12/site-packages/openff/toolkit/topology/topology.py:1785: SyntaxWarning: invalid escape sequence '\['
  smi = re.sub("\[[A-Za-z]{1,2}\]", "[*]", smi)
/home/joshmitchell/Documents/openff/openff-pdbscan/.soap/test/lib/python3.12/site-packages/openff/toolkit/utils/rdkit_wrapper.py:1670: SyntaxWarning: invalid escape sequence '\*'
  """


UnassignedChemistryInPDBError: Some bonds or atoms in the input could not be identified.

Hint: The following residues were assigned names that do not match the residue name in the input, or could not be assigned residue names at all. This may indicate that atoms are missing from the input or some other error. The OpenFF Toolkit requires all atoms, including hydrogens, to be explicit in the input to avoid ambiguities in protonation state or bond order:
    Input residue A:MET#0001 contains atoms matching substructures {'No match', 'PEPTIDE_BOND'}
    Input residue A:SER#0130 contains atoms matching substructures {'PEPTIDE_BOND', 'No match'}

Error: The following 23 atoms exist in the input but could not be assigned chemical information from the substructure library:
    Atom     0 (N) in residue A:MET#0001
    Atom     1 (CA) in residue A:MET#0001
    Atom     4 (CB) in residue A:MET#0001
    Atom     5 (CG) in residue A:MET#0001
    Atom     6 (SD) in residue A:MET#0001
    Atom     7

In [47]:
# TODO: Get all the IDs with proteins only and hydrogens and load them directly in the toolkit

<Topology; 1 chains, 130 residues, 2222 atoms, 2241 bonds>

In [8]:
ids_without_hets_with_hydrogen = (
    pl.scan_parquet("/home/joshmitchell/Downloads/pdb/coords_parquet/coords_*.parquet")
    .select(pl.col("id", "element"))
    .unique()
    .filter(pl.col("id").is_in(ids_with_hets).not_())
    .filter(pl.col("element") == "H")
    .select(pl.col("id"))
    .collect(streaming=True)
)
ids_without_hets_with_hydrogen

id
str
"""2krm"""
"""2cqr"""
"""1wjo"""
"""7qto"""
"""2k1k"""
…
"""5nca"""
"""1fra"""
"""2lvg"""


In [9]:
def series_to_list(series):
    return list(tup[0] for tup in series.iter_rows())

def ids_to_pdbs(ids: list[str]) -> list[Path]:
    pdbs = {
        path
        for path in Path("/home/joshmitchell/Downloads/pdb").glob("*/*.ent.gz")
        if path.stem[3:7] in ids
    }
    assert len(ids) == len

In [58]:
(pdbs), f"{len(ids)=}, {len(pdbs)=}"
    return list(pdbs)

topology_from_pdb_proteins_with_hydrogens = test_pdb_loader(
    pdb_loader=Topology.from_pdb,
    chunk_size=100,
    threads=24,
    paths=ids_to_pdbs(series_to_list(ids_without_hets_with_hydrogen)),
)
topology_from_pdb_proteins_with_hydrogens

batch 0
concatenating
rechunking
batch 1
concatenating
rechunking
batch 2
concatenating
rechunking
batch 3
concatenating
rechunking
batch 4
concatenating
rechunking


id,err,n_molecules,n_atoms,n_hydrogens
str,str,u64,u64,u64
"""1v60""","""UnassignedChemistryInPDBError(…",null,null,null
"""2afe""","""UnassignedChemistryInPDBError(…",null,null,null
"""2bbx""","""UnassignedChemistryInPDBError(…",null,null,null
"""2lq0""","""UnassignedChemistryInPDBError(…",null,null,null
"""2l04""","""UnassignedChemistryInPDBError(…",null,null,null
…,…,…,…,…
"""2ksg""",null,1,696,358
"""2k9u""",null,2,2109,1045
"""2ko3""",null,1,1239,639


In [59]:
topology_from_pdb_proteins_with_hydrogens.write_parquet("peptideonly_hydrogens_shippedtopfrompdb_df.parquet")

In [60]:
errors = topology_from_pdb_proteins_with_hydrogens.filter(pl.col("err").is_not_null()).select(["id", "err"])
errors

id,err
str,str
"""1v60""","""UnassignedChemistryInPDBError(…"
"""2afe""","""UnassignedChemistryInPDBError(…"
"""2bbx""","""UnassignedChemistryInPDBError(…"
"""2lq0""","""UnassignedChemistryInPDBError(…"
"""2l04""","""UnassignedChemistryInPDBError(…"
…,…
"""1wf0""","""UnassignedChemistryInPDBError(…"
"""1pi7""","""UnassignedChemistryInPDBError(…"
"""2l32""","""UnassignedChemistryInPDBError(…"


In [8]:
from tempfile import NamedTemporaryFile
from functools import partial
from openmm.app import PDBFile

def topology_via_pdbfixer(file: TextIO) -> Topology:
    from pdbfixer import PDBFixer

    fixer = PDBFixer(pdbfile=file)
    fixer.findMissingResidues()
    
    # This section removes terminal loops
    chainLengths = [len([*chain.residues()]) for chain in fixer.topology.chains()]
    for chainidx, residx in list(fixer.missingResidues):
        if residx == 0:
            fixer.missingResidues[chainidx, residx] = []
        elif residx == chainLengths[chainidx]:
            fixer.missingResidues[chainidx, residx] = []
    
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(keepWater=True)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(pH=7.4)

    with NamedTemporaryFile(suffix=".pdb") as temp_pdb:
        with open(temp_pdb.name, 'wt') as f:
            PDBFile.writeFile(fixer.topology, fixer.positions, file=f)
        topology = Topology.from_pdb(temp_pdb.name)
    return topology

In [31]:
%load_ext snakeviz

In [34]:
%%snakeviz
with gzip.open(list(protein_only_pdbs)[1], 'rt') as f:
    topology_via_pdbfixer(f)

 
*** Profile stats marshalled to file '/tmp/tmp5f0xxr72'.
Embedding SnakeViz in this document...
<function display at 0x7961c299d1c0>


In [11]:
from typing import Sequence, Iterable
import numpy as np

def subsample[T](sequence: Sequence[T], n: int, seed=None) -> Iterable[T]:
    for index in np.random.default_rng(seed).integers(low=0, high=len(sequence), size=n):
        yield sequence[index]

In [12]:
pdbfixer_pdb_loader_df = test_pdb_loader(
    pdb_loader=topology_via_pdbfixer,
    chunk_size=10,
    threads=1,
    paths=subsample(list(protein_only_pdbs), 100, seed=314),
)
pdbfixer_pdb_loader_df.write_parquet("peptideonly_pdbfixer_df.parquet")
pdbfixer_pdb_loader_df

batch 0
concatenating
rechunking
batch 1
concatenating
rechunking
batch 2
concatenating
rechunking
batch 3
concatenating
rechunking
batch 4
concatenating
rechunking
batch 5
concatenating
rechunking
batch 6
concatenating
rechunking
batch 7
concatenating
rechunking
batch 8
concatenating
rechunking
batch 9
concatenating
rechunking


id,err,n_molecules,n_atoms,n_hydrogens
str,str,u64,u64,u64
"""1s5r""",null,2,1795,883
"""1ym7""","""UnassignedChemistryInPDBError(…",null,null,null
"""7egm""","""UnassignedChemistryInPDBError(…",null,null,null
"""7xex""","""UnassignedChemistryInPDBError(…",null,null,null
"""2eem""",null,1,540,268
…,…,…,…,…
"""2kdd""",null,2,1802,930
"""1tgl""",null,1,4038,1985
"""2ly4""","""UnassignedChemistryInPDBError(…",null,null,null


In [13]:
errors = pdbfixer_pdb_loader_df.filter(pl.col("err").is_not_null()).select(["id", "err"])
errors

id,err
str,str
"""1ym7""","""UnassignedChemistryInPDBError(…"
"""7egm""","""UnassignedChemistryInPDBError(…"
"""7xex""","""UnassignedChemistryInPDBError(…"
"""6wwa""","""UnassignedChemistryInPDBError(…"
"""8hbj""","""UnassignedChemistryInPDBError(…"
…,…
"""5y4o""","""UnassignedChemistryInPDBError(…"
"""5k5g""","""UnassignedChemistryInPDBError(…"
"""6rc8""","""UnassignedChemistryInPDBError(…"


In [16]:
pdbfixer_pdb_loader_df.filter(pl.col("err").str.starts_with("UnassignedChemistryInPDBError").not_())

id,err,n_molecules,n_atoms,n_hydrogens
str,str,u64,u64,u64
"""7n9u""","""KeyError('UNK')""",null,null,null


In [ ]:
try:
    assert False
except Exception as e:
    

In [3]:
import MDAnalysis.topology.tables

MDAnalysis.topology.tables.vdwradii

/home/joshmitchell/Documents/openff/openff-pdbscan/.soap/test/lib/python3.12/site-packages/MDAnalysis/topology/tables.py:52: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.topology.tables has been moved to MDAnalysis.guesser.tables. This import point will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)


{'H': 1.1,
 'HE': 1.4,
 'LI': 1.82,
 'BE': 1.53,
 'B': 1.92,
 'C': 1.7,
 'N': 1.55,
 'O': 1.52,
 'F': 1.47,
 'NE': 1.54,
 'NA': 2.27,
 'MG': 1.73,
 'AL': 1.84,
 'SI': 2.1,
 'P': 1.8,
 'S': 1.8,
 'CL': 1.75,
 'AR': 1.88,
 'K': 2.75,
 'CA': 2.31,
 'NI': 1.63,
 'CU': 1.4,
 'ZN': 1.39,
 'GA': 1.87,
 'GE': 2.11,
 'AA': 1.85,
 'SE': 1.9,
 'BR': 1.85,
 'KR': 2.02,
 'RR': 3.03,
 'SR': 2.49,
 'PD': 1.63,
 'AG': 1.72,
 'CD': 1.58,
 'IN': 1.93,
 'SN': 2.17,
 'SB': 2.06,
 'TE': 2.06,
 'I': 1.98,
 'XE': 2.16,
 'CS': 3.43,
 'BA': 2.68,
 'PT': 1.75,
 'AU': 1.66,
 'HH': 1.55,
 'TL': 1.96,
 'PB': 2.02,
 'BI': 2.07,
 'PO': 1.97,
 'AT': 2.02,
 'RN': 2.2,
 'FR': 3.48,
 'RA': 2.83,
 'U': 1.86}

In [4]:
import MDAnalysis as mda
import mda.topology.tables

ModuleNotFoundError: No module named 'mda'